In [83]:
import ecc
from ecc import PrivateKey,S256Point
from random import randint
import hashlib
from helper import little_endian_to_int

def double_sha256(s):
    return hashlib.sha256(hashlib.sha256(s).digest()).digest()

In [84]:
#z = randint(0, 2**256).to_bytes(256//8, 'little')
my_str = "Rima Rana Seth Lee"
z = str.encode(my_str)

In [85]:
print(z)

b'Rima Rana Seth Lee'


In [86]:
print(z.hex())

52696d612052616e612053657468204c6565


In [5]:
# hash
def H(*args):
    return double_sha256(b''.join(args))

# int(hash)
def HI(*args):
    return little_endian_to_int(H(*args))

#Schnorr
def sign(pk, z):
        k = randint(0, 2**256) # nonce
        R = k * ecc.G          # nonce point
        s = (k + HI(R.sec(), z) * pk.secret)%ecc.N
        return R, s

def verify(R, s, z):
        Q = s * ecc.G - HI(R.sec(),z) * point
        return Q == R


In [8]:
pk = PrivateKey(randint(0, 2**256))
point = pk.point # public point

In [9]:
R, s = sign(pk, z)
print(verify(R,s,z))
assert verify(R, s, z)

True


In [10]:
#Ns is number of inputs, for our problem its 3
Ns = 3

pks = [PrivateKey(randint(0, 2**256)) for _ in range(Ns)]
points = [pk.point for pk in pks]

In [11]:
#BN
def sign(pks, z):
        ks = [randint(0, 2**256) for _ in range(len(pks))]

        L = H(*[p.sec() for p in points])
        Rs = [k*ecc.G for k in ks]
        R = sum(Rs, ecc.S256Point(None,None))
        cs = [HI(L,p.sec(),R.sec(),z) for p in points]
        s = sum(k + c*pk.secret for k,c,pk in zip(ks,cs,pks))%ecc.N
        return R, s

def verify(R, s, z):
        L = H(*[p.sec() for p in points])
        cs = [HI(L,p.sec(),R.sec(),z) for p in points]
        return R == s*ecc.G - sum((c*p for c,p in zip(cs, points)),ecc.S256Point(None,None))

In [12]:
R, s = sign(pks, z)
print(verify(R,s,z))
assert verify(R, s, z)

True


In [62]:
#Mu
#For signing we will provide, aggregate Public key P and Signature (R,s)
def sign(pks, z):
        L = H(*[p.sec() for p in points])
        ks = [randint(0, 2**256) for _ in range(len(pks))]
        Rs = [k*ecc.G for k in ks]
        R = sum(Rs, ecc.S256Point(None,None))
        # aggregate key
        P = sum((HI(L, p.sec())*p for p in points), ecc.S256Point(None,None))
        cs = [HI(R.sec(),z)*HI(L,p.sec()) for p in points]
        s = sum([k+c*pk.secret for k,c,pk in zip(ks,cs,pks)]) % ecc.N
        return R, s, P

#We verify by using all three
def verify(R, s, z, P):
        return R == s*ecc.G - HI(R.sec(),z)*P

   

In [80]:
R, s, P = sign(pks, z)
print(R.sec().hex())
print("###################")
print(s)
print("###################")
print(P.sec().hex())

0306e1653a8253ff2bebf00a200998c2bfbf1233c9cd2c9c1b89280f585f306fd0
###################
100122388257161252664283364891666006783378674313668849586148163489512916853636
###################
034b4619f4635c71e2e5f58a9c9c704df9fe7219ac1f727c55ced674a936e293b5


In [82]:
R_sec = bytes.fromhex('0306e1653a8253ff2bebf00a200998c2bfbf1233c9cd2c9c1b89280f585f306fd5')
# parse the sec format to get the public key
R_point = S256Point.parse(R_sec)
s_n = bytes.fromhex('100122388257161252664283364891666006783378674313668849586148163489512916853636')
P_sec = bytes.fromhex('034b4619f4635c71e2e5f58a9c9c704df9fe7219ac1f727c55ced674a936e293b5')
P_point = S256Point.parse(P_sec)

print(verify(R_point, s, z, P_point))
#assert verify(R, s, z, P)

False
